# 『2022信通院兴智杯：深度学习模型可解释竞赛』- 文本相似度可解释性评测
## 1、项目介绍
深度学习模型在很多NLP任务上已经取得巨大成功，但其常被当作一个黑盒使用，内部预测机制对使用者是不透明的。这使得深度学习模型结果不被使用者信任，增加了落地难度，尤其在医疗、法律等特殊领域。同时，当模型出现效果不好或鲁棒性差等问题时，由于不了解其内部机制，很难对模型进行改进优化。近期，深度学习模型的可解释性被越来越多的人关注。但模型的可解释性评估还不够完善，本基线提供了文本相似度任务的评测数据和相关评测指标，旨在评估模型的可解释性。
## 2、基线运行


### 依赖安装
安装一些必须的依赖包。

In [ ]:
!pip3 install paddlepaddle-gpu
!pip3 install -U paddlenlp==2.3.0

### 数据准备
#### 1）模型训练数据
我们推荐使用LCQMC数据集训练中文相似度计算模型。Paddlenlp框架会自动下载及缓存训练数据集，默认缓存存储路径为"~/.paddlenlp/datasets"。如需修改训练数据，请参考『初始化工作』中DATASET_NAME的修改。
#### 2）下载预训练模型
基线使用了ERNIE-3.0-base预训练模型。Paddlenlp框架自动缓存模型文件，默认缓存存储路径为"~/.paddlenlp/models"。如需修改依赖的预训练模型，请在『初始化工作』中修改MODEL_NAME。

### 初始化工作
初始化工作包括了模型选择及加载、训练数据集选择、模型存储路径设定、抽取证据的长度占原文本长度的比例设定等。可按需更改。

In [ ]:
import sys
import json
import numpy as np
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer
sys.path.append("..")
sys.path.append("../../")

# Select pre-trained model
MODEL_NAME = "ernie-3.0-base-zh" # choose from ["ernie-1.0", "ernie-1.0-base-zh", "ernie-1.0-large-zh-cw", "ernie-2.0-base-zh", "ernie-2.0-large-zh", "ernie-3.0-xbase-zh", "ernie-3.0-base-zh", "ernie-3.0-medium-zh", "ernie-3.0-mini-zh", "ernie-3.0-micro-zh", "ernie-3.0-nano-zh"]
# Select dataset for model training
DATASET_NAME = 'lcqmc'
# Set the path to save the trained model
MODEL_SAVE_PATH = f'../assets/{DATASET_NAME}-{MODEL_NAME}'
# Set the rationale length ratio which determines the length of the extracted rationales.
RATIONALE_RATIO = 0.7


# Init model and tokenizer
model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

### 模型训练
这里以ERNIE-3.0为例训练一个文本相似度模型。

In [ ]:
from paddlenlp.datasets import load_dataset
from assets.utils import training_model

# Load dataset
train_ds, dev_ds, test_ds = load_dataset(DATASET_NAME, splits=["train", "dev", "test"])

# Start training
training_model(model, tokenizer, train_ds, dev_ds, save_dir=MODEL_SAVE_PATH)

### 重要度分数获取
该步为输入中每个词赋一个重要度分数，表示该词对预测的影响度。重要度分数获取共分三步。
#### 1）加载模型和评测数据集
更改模型以及评估数据的存储路径（MODEL_PATH和DATA_PATH），完成模型和评测数据集的加载。赛段一数据量为1712条，赛段二数据量为4167条，请确认评测数据集完整。

In [ ]:
from assets.utils import load_data

# Correct MODEL_PATH and DATA_PATH before executing
MODEL_PATH = MODEL_SAVE_PATH + '/model_state.pdparams'
DATA_PATH = '/home/aistudio/TrustAI/tutorials/assets/sim_interpretation_A.txt'

# Load the trained parameters
state_dict = paddle.load(MODEL_PATH)
model.set_dict(state_dict)

# Load test data
data = load_data(DATA_PATH)
print("Num of data:", len(data))


#### 2）数据预处理

a) 输入格式化：将输入的两个文本组织成模型预测所需格式，如对于Ernie3.0-base模型，其输入形式为[CLS]query[SEP]title[SEP]

b) 分词位置索引：计算每个分词结果对应的原文位置索引，这里的分词包括模型分词和标准分词

In [7]:
from trustai.interpretation import get_word_offset

# Add CLS and SEP tags to both original text and standard splited tokens
contexts = []
standard_split = []
for idx in data:
    example = data[idx]
    contexts.append("[CLS]" + example['query'] + "[SEP]" + example['title'] + "[SEP]")
    standard_split.append(["[CLS]"] + example['text_q_seg'] + ["[SEP]"] + example['text_t_seg'] + ["[SEP]"])

# Get the offset map of tokenized tokens and standard splited tokens
ori_offset_maps = []
standard_split_offset_maps = []
for i in range(len(contexts)):
    ori_offset_maps.append(tokenizer.get_offset_mapping(contexts[i]))
    standard_split_offset_maps.append(get_word_offset(contexts[i], standard_split[i]))

#### 3）重要度分数获取
我们提供attention，IG，LIME等三种解释方法，可根据实际实验结果选取最有效的一种方法。

##### a） Attention-based Interpreter

In [8]:
from trustai.interpretation.token_level.common import attention_predict_fn_on_paddlenlp
from trustai.interpretation.token_level import AttentionInterpreter
from assets.utils import create_dataloader_from_scratch
# Hyperparameters
BATCH_SIZE = 8

# Init an attention interpreter and get the importance scores
att = AttentionInterpreter(model, device="gpu", predict_fn=attention_predict_fn_on_paddlenlp)

# Use attention interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += att(batch)
    else:
        interp_results = att(batch)

# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = att.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

##### b）IG-based Interpreter

In [ ]:
from trustai.interpretation.token_level import IntGradInterpreter
from assets.utils import create_dataloader_from_scratch
# Hyperparameters
IG_STEP = 100
BATCH_SIZE = 8

# Init an IG interpreter
ig = IntGradInterpreter(model, device="gpu")

# Use IG interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += ig(batch, steps=IG_STEP)
    else:
        interp_results = ig(batch, steps=IG_STEP)

# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = ig.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

##### c）LIME Interpreter

In [4]:

from trustai.interpretation.token_level import LIMEInterpreter
from assets.utils import create_dataloader_from_scratch
# Hyperparameters
LIME_SAMPLES = 1000
BATCH_SIZE = 8

# Init an LIME interpreter
lime = LIMEInterpreter(model, device="gpu",
    unk_id=tokenizer.convert_tokens_to_ids('[UNK]'),
    pad_id=tokenizer.convert_tokens_to_ids('[PAD]'))

# Use LIME interpreter to get the importance scores for all data
interp_results = None
for batch in create_dataloader_from_scratch(list(data.values()), tokenizer, BATCH_SIZE):
    if interp_results:
        interp_results += lime(batch, num_samples=LIME_SAMPLES)
    else:
        interp_results = lime(batch, num_samples=LIME_SAMPLES)
    
# Align the results back to the standard splited tokens so that it can be evaluated correctly later
align_res = lime.alignment(interp_results, contexts, standard_split, standard_split_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'])

### 生成用于评估的数据
评估文件格式要求是4列数据：编号\t预测标签\t证据1\t证据2，我们提供了脚本将模型输出结果转成评估所需格式。

In [9]:
import math

# Re-sort the token index according to their importance scores
def resort(index_array, importance_score):
    res = sorted([[idx, importance_score[idx]] for idx in index_array], key=lambda x:x[1], reverse=True)
    res = [n[0] for n in res]
    return res

# Post-prepare the result data so that it can be used for the evaluation directly
def prepare_eval_data(data, results, paddle_model):
    res = {}
    for data_id, inter_res in zip(data, results):
        # Split importance score vectors for query and title from inter_res.word_attributions
        query_importance_score = np.array(inter_res.word_attributions[1:len(data[data_id]['text_q_seg'])+1])
        title_importance_score = np.array(inter_res.word_attributions[len(data[data_id]['text_q_seg'])+2:-1])
        # Extract topK importance scores
        query_topk = math.ceil(len(data[data_id]['text_q_seg'])*RATIONALE_RATIO)
        title_topk = math.ceil(len(data[data_id]['text_t_seg'])*RATIONALE_RATIO)
        
        eval_data = {}        
        eval_data['id'] = data_id
        eval_data['pred_label'] = inter_res.pred_label
        # Find the token index of the topK importance scores
        eval_data['rationale_q'] = np.argpartition(query_importance_score, -query_topk)[-query_topk:]
        eval_data['rationale_t'] = np.argpartition(title_importance_score, -title_topk)[-title_topk:]
        # Re-sort the token index according to their importance scores
        eval_data['rationale_q'] = resort(eval_data['rationale_q'], query_importance_score)
        eval_data['rationale_t'] = resort(eval_data['rationale_t'], title_importance_score)

        res[data_id] = eval_data
    return res

# Generate results for evaluation
predicts = prepare_eval_data(data, align_res, model)
out_file = open('./sim_rationale.txt', 'w')
for key in predicts:
    out_file.write(str(predicts[key]['id'])+'\t'+ str(predicts[key]['pred_label'])+'\t')
    for idx in predicts[key]['rationale_q'][:-1]:
        out_file.write(str(idx)+',')
    out_file.write(str(predicts[key]['rationale_q'][-1])+'\t')

    for idx in predicts[key]['rationale_t'][:-1]:
        out_file.write(str(idx)+',')
    out_file.write(str(predicts[key]['rationale_t'][-1])+'\n')
out_file.close()